In [1]:
!pip install bs4

In [2]:
!pip install requests

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings

In [13]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0',
    'Accept': 'application/json',
}

data = []
page = 0
count = 16

In [15]:
warnings.filterwarnings("ignore", category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

article_number = 0

while count < 500:
    url = f"https://bangla.dhakatribune.com/api/theme_engine/get_ajax_contents?widget=622&start={page}&count={count}&page_id=1204&subpage_id=0&author=0&tags=&archive_time=&filter="

    try:
        response = requests.get(url, headers=headers, verify=False)
        response.raise_for_status()

        soup = BeautifulSoup(response.json().get("html", ""), 'html.parser')

        articles = soup.select('div.each.col_in')

        if not articles:
            break

        for idx, article in enumerate(articles, start=1):
            title = article.find('h2', class_='title').text.strip()
            link = f"https:{article.find('a', class_='link_overlay')['href']}"

            content_response = requests.get(link, headers=headers, verify=False)
            content_response.raise_for_status()

            content_soup = BeautifulSoup(content_response.content, 'html.parser')

            summary = content_soup.find('div', class_='content_highlights jw_detail_content_holder content mb16')
            summary_text = '। '.join([item.text.strip() for item in summary.find_all(['p', 'li'])]) if summary else None

            article_body = content_soup.find('div', class_='jw_article_body')
            article_text = ' '.join([p.text.strip() for p in article_body.find_all('p')]) if article_body else None

            article_number += 1

            if summary_text is not None or article_text is not None:
                data.append({'URL': link, 'Headline': title, 'Summary': summary_text, 'Article': article_text})
                print(f"Article {article_number}: {link}")

        page += count

    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
        break

df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/dhaka_tribune_bangla.csv', index=False, encoding='utf-8')

An error occurred: 403 Client Error: Forbidden for url: https://bangla.dhakatribune.com/api/theme_engine/get_ajax_contents?widget=622&start=0&count=16&page_id=1204&subpage_id=0&author=0&tags=&archive_time=&filter=
